In [6]:
import yfinance as yf
import plotly.express as px
import plotly.io as pio
import numpy as np
import pyarrow
import pandas as pd

In [7]:
combined_df = pd.read_parquet('/Users/paigeblackstone/Desktop/Portfolio29/Portfolio29/Investment_Management/datapull_YF_081224/congress_stock_data.parquet')

In [8]:
combined_df.columns.to_list()

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Volume',
 'Ticker',
 'Industry',
 'Sector',
 'Market Cap',
 'PE Ratio',
 'Price to Book',
 'Return on Equity',
 'Price to Sales',
 'Dividend Yield',
 'Current Ratio',
 'Beta']

In [9]:
combined_df['open_close_diff'] = ((combined_df['Close'] - combined_df['Open']) / combined_df['Open']) * 100

In [10]:
combined_df = combined_df.sort_values(by=['Ticker', 'Date'])

# Group by 'Ticker' and calculate the month-over-month percentage change in 'Volume'
combined_df['volume_mom_change'] = combined_df.groupby('Ticker')['Volume'].pct_change() * 100

In [17]:
combined_df['Date'] = pd.to_datetime(combined_df['Date'], utc=True)
combined_df = combined_df.sort_values(by=['Ticker', 'Date'])
combined_df.set_index('Date', inplace=True)
monthly_close = combined_df.groupby('Ticker').resample('M').agg({'Close': 'last'})
monthly_close.reset_index(inplace=True)
monthly_close['monthly_return'] = monthly_close.groupby('Ticker')['Close'].pct_change() * 100


In [18]:
combined_df = pd.merge(combined_df.reset_index(), monthly_close[['Ticker', 'Date', 'monthly_return']], on=['Ticker', 'Date'], how='left')


In [19]:
print(combined_df.head())

                       Date        Open        High         Low       Close  \
0 2020-01-31 00:00:00+00:00  216.773911  223.873828  200.863791  216.249698   
1 2020-02-29 00:00:00+00:00  207.110925  219.244956  181.605603  207.606898   
2 2020-03-30 23:00:00+00:00  165.829335  197.213635  137.178695  165.552751   
3 2020-04-29 23:00:00+00:00  176.624813  209.125851  150.532978  176.314108   
4 2020-05-30 23:00:00+00:00  215.125735  240.377989  198.568780  216.181845   

    Volume   Ticker                        Industry                  Sector  \
0  1707831  0QZI.IL  Internet Content & Information  Communication Services   
1   917405  0QZI.IL  Internet Content & Information  Communication Services   
2  1514934  0QZI.IL  Internet Content & Information  Communication Services   
3  1279808  0QZI.IL  Internet Content & Information  Communication Services   
4   753312  0QZI.IL  Internet Content & Information  Communication Services   

     Market Cap  PE Ratio  Price to Book  Return o

In [ ]:
def risk_return(df, adj=True, crypto=False):
    if adj:
        close=df.loc[:, "adj_close"].copy()
    else:
        close=df.loc[:, "close"].copy()
    if crypto:
        trading_days=365
    else: 
        trading_days=252
    
    stock_returns=close.pct_change().dropna().mul(100)
    summary=stock_returns.describe().T.loc[:, ["mean", "std"]]
    summary["mean"]=summary["mean"]*trading_days
    summary["std"]=summary["std"]*np.sqrt(trading_days)
    summary.rename(columns={'mean':'Return', 'std': 'Risk'}, inplace=True)
    fig=px.scatter(summary,
                   x='Risk',
                   y='Return',
                   title='monthly risk/return',
                   text=summary.index,
                   template='plotly_dark')
    fig.update_traces(marker={'size':12},
                      textposition='top center')
    fig.update_layout(
        legend=dict(title=None),
        title={
            'y':0.9,
            'x':0.5,
            'font':{'size':20},
            'xanchor':'center',
            'yanchor':'top'
        },
        xaxis=dict(title=dict(font=dict(size=14))),
        yaxis=dict(title=dict(font=dict(size=14)))
    )
    fig.show()


In [ ]:
def return_corr(df, adj=True, crypto=False):
    if adj:
        close=df.loc[:, "adj_close"].copy()
    else:
        close=df.loc[:, "close"].copy()
    
    if crypto:
        trading_days=365
    else:
        trading_days=252
    
    stock_return=close.pct_change().dropna().mul(100)
    
    summary=stock_return.describe().T.loc[:, ["mean", "std"]]
    summary["mean"]=summary["mean"]*trading_days
    summary["std"]=summary["std"]*np.sqrt(trading_days)
    summary.rename(columns={'mean':'Return', 'std': 'Risk'}, inplace=True)
    
    fig=px.imshow(stock_return.corr(), text_auto=True, color_continuous_scale='tempo', template='plotly_dark', title="correlation of returns")
    fig.update_layout(
        legend=dict(title=None),
        title={
            'y':0.9,
            'x':0.5,
            'font':{'size':20},
            'xanchor':'center',
            'yanchor':'top'
        })
    fig.show()